<a href="https://colab.research.google.com/github/bo7/Coursera_Capstone/blob/master/Coursera_Capstone_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Add libraries</h3>



In [0]:
import pandas as pd
import requests
import io
from bs4 import BeautifulSoup


<h3>Read table from Wikipedia in pandas data frame</h3>

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

<h3>Clean and amend data frame</h3>
<ol>
  <li>remove not assigned boroughs</li>
  <li>join postcodes with multiple boroughs in one comma seperated row</li>
  <li>copy not assigned neighbourhoods to borough</li>
</ol>

In [0]:
# Clean Not assigned Boroughs  
df = df[~df['Borough'].isin(['Not assigned'])]
# Join grouped Boroughs to one comma seperated line
df =df.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(','.join)
# Boroughs with no neighbourhood get the borough itself as neighbourhood
df.Neighbourhood = df.Borough.where(df.Neighbourhood == 'Not assigned', df.Neighbourhood)

<h3>Print head of data frame and shape</h3>

In [7]:
print(df.head())
print(df.shape)

  Postcode           Borough                    Neighbourhood
0      M3A        North York                        Parkwoods
1      M4A        North York                 Victoria Village
2      M5A  Downtown Toronto         Harbourfront,Regent Park
3      M6A        North York  Lawrence Heights,Lawrence Manor
4      M7A      Queen's Park                     Queen's Park
(103, 3)


<h3>Load Geospatial data to df</h3>

In [0]:
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
df2=pd.read_csv(io.StringIO(s.decode('utf-8')))

<h3>Inner Join of data frames on Postcode</h3>

In [0]:
df2.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df =pd.merge(df, df2, on='Postcode', how='inner')


<h3>Check head and shape</h3>

In [30]:
print(df.head())
df.shape

  Postcode           Borough  ...   Latitude  Longitude
0      M3A        North York  ...  43.753259 -79.329656
1      M4A        North York  ...  43.725882 -79.315572
2      M5A  Downtown Toronto  ...  43.654260 -79.360636
3      M6A        North York  ...  43.718518 -79.464763
4      M7A      Queen's Park  ...  43.662301 -79.389494

[5 rows x 5 columns]


(103, 5)